In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [3]:
data = pd.read_csv('../../dataset (missing + split)/train.csv', skip_blank_lines=True)

#We create the interquartile range to delete the noise from the vars and then recreate our new data set without noise

In [4]:
#Let's create a column for the data set of all zeros to fill with the outliers
data['outliers'] = 0
listofscales = [7,4.4,3.4]
dfs = {}

#1% = 7,  5% = 4.4,  10% = 3.4

In [5]:
for l in listofscales:  
    df_number = f'df_{l}'
    key = f'{"new_data"}_{l}'  
    data_basis = data.copy()
    for x in data.columns:
        if np.issubdtype(data[x].dtype, np.number) and len(data[x].unique()) > 5:
            
            #Handling missing values to check for changes
            #data[x].fillna(data[x].mean())
            #data.dropna(subset=[x], inplace=True)
            #Interquartile range
            Q1 = np.percentile(data[x], 25)
            Q3 = np.percentile(data[x], 75)
            IQR = Q3 - Q1

            #Lower and upper bound for outliers
            lower_bound = Q1 - l * IQR
            upper_bound = Q3 + l * IQR

            data.loc[((data[x] < lower_bound) | (data[x] > upper_bound)),'outliers'] = 1
        #new_data = data[data['outliers']!= 1]
    dfs[key] = pd.DataFrame(data)
    data = data_basis


In [16]:
for key, df in dfs.items():
    print(f"New Data: {key}")
    #print(df)
    print(df['outliers'].unique())
    print(df.groupby('outliers').size().reset_index(name='Frequency'))

New Data: new_data_7
[0]
   outliers  Frequency
0         0      14842
New Data: new_data_4.4
[0]
   outliers  Frequency
0         0      14293
New Data: new_data_3.4
[0]
   outliers  Frequency
0         0      13499


In [15]:
for key, df in dfs.items(): 
    no_outliers = df[df['outliers']!= 1]
    dfs[key] = pd.DataFrame(no_outliers)

In [7]:
#Creating the new df
new_data = data[data['outliers']!= 1]
new_data = pd.DataFrame(new_data)

#The count of outliers in total is really high, it's almost a third of the data set, in this case we have various options.

1. winsorizing(Truncating or Capping)
2. trimming(Delete them)
3. trying to eliminate outliers after the feature selection and see if the number it's lower

#Check the difference in boxplots and hist when we change the IQR scale

#Hist code

In [38]:
""" numeric_columns = data[['duration_ms']]
numeric_columns = data.select_dtypes(include=['number']).columns
for col in numeric_columns:
        for key, df in dfs.items():
                print(col+'_'+str(key))
                bins_number = math.ceil(math.log2(df[col].size)) + 1
                df[col].hist(bins = bins_number, color = 'c', edgecolor = 'b')
                mean_value = df[col].mean()
                #mode_value = master[col].mode()
                median_value = df[col].median()
                plt.ticklabel_format(style='plain', axis='x')
                plt.title(str.capitalize(col) +'-'+ key + ' Histogram')
                plt.xlabel(str.capitalize(col))
                plt.ylabel('Frequency')
                plt.axvline(mean_value, color = 'blue', linestyle = 'dashed')
                #plt.axvline(mode_value[0], color='#FF00FF', linestyle = '--')
                plt.axvline(median_value, color = 'red', linestyle = '--')
                plt.show() """

" numeric_columns = data[['duration_ms']]\nnumeric_columns = data.select_dtypes(include=['number']).columns\nfor col in numeric_columns:\n        for key, df in dfs.items():\n                print(col+'_'+str(key))\n                bins_number = math.ceil(math.log2(df[col].size)) + 1\n                df[col].hist(bins = bins_number, color = 'c', edgecolor = 'b')\n                mean_value = df[col].mean()\n                #mode_value = master[col].mode()\n                median_value = df[col].median()\n                plt.ticklabel_format(style='plain', axis='x')\n                plt.title(str.capitalize(col) +'-'+ key + ' Histogram')\n                plt.xlabel(str.capitalize(col))\n                plt.ylabel('Frequency')\n                plt.axvline(mean_value, color = 'blue', linestyle = 'dashed')\n                #plt.axvline(mode_value[0], color='#FF00FF', linestyle = '--')\n                plt.axvline(median_value, color = 'red', linestyle = '--')\n                plt.show() "

#Boxplot code

In [9]:
numeric_columns = new_data.select_dtypes(include=['number']).columns
for col in numeric_columns:
        fig, ax = plt.subplots()
        boxprops = dict(linestyle='--', linewidth=2, color='blue', edgecolor='red')
        flierprops = dict(marker='o', markerfacecolor='green', markersize=8, linestyle='none')
        medianprops = dict(linestyle='-', linewidth=2, color='orange')
        box = ax.boxplot(new_data[col], patch_artist=True, boxprops=boxprops, flierprops=flierprops, medianprops=medianprops)
        ax.yaxis.grid(True, linestyle='--', which='major', color='gray', alpha=0.7)
        ax.axhline(y=0, color='gray', linestyle='--', linewidth=2)
        plt.boxplot(new_data[col])
        plt.title(str.capitalize(col) +' Box Plot')
        plt.ylabel('Value')
        plt.show()

" numeric_columns = new_data.select_dtypes(include=['number']).columns\nfor col in numeric_columns:\n        fig, ax = plt.subplots()\n        boxprops = dict(linestyle='--', linewidth=2, color='blue', edgecolor='red')\n        flierprops = dict(marker='o', markerfacecolor='green', markersize=8, linestyle='none')\n        medianprops = dict(linestyle='-', linewidth=2, color='orange')\n        box = ax.boxplot(new_data[col], patch_artist=True, boxprops=boxprops, flierprops=flierprops, medianprops=medianprops)\n        ax.yaxis.grid(True, linestyle='--', which='major', color='gray', alpha=0.7)\n        ax.axhline(y=0, color='gray', linestyle='--', linewidth=2)\n        plt.boxplot(new_data[col])\n        plt.title(str.capitalize(col) +' Box Plot')\n        plt.ylabel('Value')\n        plt.show() "